## Importing Libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import os

print(tf.__version__)
print(keras.__version__)

1.13.1
2.2.4-tf


## Enable Eager Mode

In [2]:
tf.enable_eager_execution()

## Hyper Parameters

In [3]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

tf.set_random_seed(777)

## Creating a Checkpoint Directory

In [4]:
cur_dir = os.getcwd()
ckpt_dir_name = 'checkpoints'
model_dir_name = 'minst_cnn_seq'

checkpoint_dir = os.path.join(cur_dir, ckpt_dir_name, model_dir_name)
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, model_dir_name)

## MNIST/Fashion MNIST Data

In [5]:
## MNIST Dataset #########################################################
mnist = keras.datasets.mnist
class_names = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
##########################################################################

## Fashion MNIST Dataset #################################################
#mnist = keras.datasets.fashion_mnist
#class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
##########################################################################

## Datasets

In [6]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()    
    
train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.
train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
    
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)    
    
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(
                buffer_size=100000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

## Model Function

In [7]:
def create_model():
    model = keras.Sequential()
    model.add(keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu, padding='SAME', 
                                  input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu, padding='SAME'))
    model.add(keras.layers.MaxPool2D(padding='SAME'))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Dense(10))
    return model

In [8]:
model = create_model()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxP

## Loss Function

In [9]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=logits, labels=labels))    
    return loss   

## Calculating Gradient

In [10]:
def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

## Caculating Model's Accuracy

In [11]:
def evaluate(model, images, labels):
    logits = model(images, training=False)
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

## Optimizer

In [12]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

## Training

In [13]:
# train my model
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_loss = 0.
    avg_train_acc = 0.
    avg_test_acc = 0.
    train_step = 0
    test_step = 0
    
    for images, labels in train_dataset:
        grads = grad(model, images, labels)                
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = evaluate(model, images, labels)
        avg_loss = avg_loss + loss
        avg_train_acc = avg_train_acc + acc
        train_step += 1
    avg_loss = avg_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in test_dataset:        
        acc = evaluate(model, images, labels)        
        avg_test_acc = avg_test_acc + acc
        test_step += 1    
    avg_test_acc = avg_test_acc / test_step    

    print('Epoch:', '{}'.format(epoch + 1), 'loss =', '{:.8f}'.format(avg_loss), 
          'train accuracy = ', '{:.4f}'.format(avg_train_acc), 
          'test accuracy = ', '{:.4f}'.format(avg_test_acc))

print('Learning Finished!')

Learning started. It takes sometime.
Epoch: 1 loss = 0.17316563 train accuracy =  0.9588 test accuracy =  0.9839
Epoch: 2 loss = 0.04697834 train accuracy =  0.9896 test accuracy =  0.9900
Epoch: 3 loss = 0.03077060 train accuracy =  0.9937 test accuracy =  0.9922
Epoch: 4 loss = 0.02234057 train accuracy =  0.9954 test accuracy =  0.9934
Epoch: 5 loss = 0.01881780 train accuracy =  0.9965 test accuracy =  0.9934
Epoch: 6 loss = 0.01520850 train accuracy =  0.9975 test accuracy =  0.9938
Epoch: 7 loss = 0.01177239 train accuracy =  0.9982 test accuracy =  0.9929
Epoch: 8 loss = 0.01018523 train accuracy =  0.9984 test accuracy =  0.9937
Epoch: 9 loss = 0.00915931 train accuracy =  0.9989 test accuracy =  0.9928
Epoch: 10 loss = 0.00751588 train accuracy =  0.9991 test accuracy =  0.9924
Epoch: 11 loss = 0.00621627 train accuracy =  0.9992 test accuracy =  0.9914
Epoch: 12 loss = 0.00558107 train accuracy =  0.9995 test accuracy =  0.9920
Epoch: 13 loss = 0.00610260 train accuracy =  0.

## Saving Weights

In [14]:
model.save_weights(checkpoint_prefix)

Instructions for updating:
Use tf.train.CheckpointManager to manage checkpoints rather than manually editing the Checkpoint proto.


## Calculating Average Accuracy

In [15]:
def avg_accuracy(model, dataset):
    avg_acc = 0.
    step = 0
    for images, labels in dataset:        
        acc = evaluate(model, images, labels)        
        avg_acc += acc
        step += 1    
    avg_acc = avg_acc / step
    return avg_acc.numpy()

## Creating a New Model

In [16]:
new_model = create_model()

## Test Accuracy before Restore

In [17]:
avg_accuracy(new_model, test_dataset)

0.07360001

## Restore Weights

In [18]:
new_model.load_weights(checkpoint_prefix)

## Test Accuracy after Restore

In [19]:
avg_accuracy(new_model, test_dataset)

0.9935